# This notebook is for extracting the climatic features, calculated as the average over the entire period of the dataset, for the areas where wildfires have happended in Portugal.
## The source of the data is Mesogeos datacube.
## Datacube Source: https://github.com/Orion-AI-Lab/mesogeos/tree/main
## Portugal Shapefile Source: https://geodata.lib.utexas.edu/catalog/stanford-tq087tk3612
## Created by Aser Atawya for wildfire project as part of Beculer Lab at University of Lausanne


# Imports and Installing dependencies (dependencies list is not minimal)


In [ ]:
!pip install zarr
!pip install sklearn-som
!pip install minisom
!pip install rasterio
!pip install rioxarray
!pip install geojson
!pip install netCDF4

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
from sklearn.preprocessing import MinMaxScaler, scale
from minisom import MiniSom
from sklearn_som.som import SOM
from sklearn import datasets
import numpy as np
from osgeo import gdal
import rasterio as rio
import geopandas
import rasterio.features
import rioxarray
import geojson
import pandas as pd
from datetime import date, timedelta, datetime
from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_samples, silhouette_score
import math
pd.set_option('display.max_columns', None)

import scipy.stats as stats
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from shapely.geometry import mapping
from shapely.ops import cascaded_union
gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Accessing Data

In [ ]:
ds = xr.open_zarr('/content/drive/MyDrive/mesogeos/mesogeos_cube.zarr/', decode_coords=all)

In [ ]:
ds

<xarray.Dataset>
Dimensions:               (y: 1752, x: 4714, time: 6026)
Coordinates:
  * time                  (time) datetime64[ns] 2006-04-01 ... 2022-09-29
  * x                     (x) float64 -10.72 -10.71 -10.7 ... 36.72 36.73 36.74
  * y                     (y) float64 47.7 47.69 47.68 ... 30.09 30.08 30.07
Data variables: (12/30)
    aspect                (y, x) float32 dask.array<chunksize=(1752, 4714), meta=np.ndarray>
    burned_areas          (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>
    curvature             (y, x) float32 dask.array<chunksize=(1752, 4714), meta=np.ndarray>
    d2m                   (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>
    dem                   (y, x) float32 dask.array<chunksize=(1752, 4714), meta=np.ndarray>
    ignition_points       (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>
    ...                    ...
    spatial_ref           int64 ...
    ssrd                  (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>
    t2m                   (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>
    tp                    (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>
    wind_direction        (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>
    wind_speed            (time, y, x) float32 dask.array<chunksize=(1, 1752, 4714), meta=np.ndarray>

In [ ]:
# slice a rectangle including Portugal from original dataset to speeden masking
# Those points represent the eastmost, westmost, northmost, and southmost points in Portugal mainland
ds_portugal = ds.sel(x=slice(-9.50,-6.20), y=slice( 42.16, 36.96))

In [ ]:
ds_portugal

<xarray.Dataset>
Dimensions:               (y: 516, x: 327, time: 6026)
Coordinates:
  * time                  (time) datetime64[ns] 2006-04-01 ... 2022-09-29
  * x                     (x) float64 -9.491 -9.481 -9.471 ... -6.218 -6.208
  * y                     (y) float64 42.15 42.14 42.13 ... 36.99 36.98 36.97
Data variables: (12/30)
    aspect                (y, x) float32 dask.array<chunksize=(516, 327), meta=np.ndarray>
    burned_areas          (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>
    curvature             (y, x) float32 dask.array<chunksize=(516, 327), meta=np.ndarray>
    d2m                   (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>
    dem                   (y, x) float32 dask.array<chunksize=(516, 327), meta=np.ndarray>
    ignition_points       (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>
    ...                    ...
    spatial_ref           int64 ...
    ssrd                  (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>
    t2m                   (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>
    tp                    (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>
    wind_direction        (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>
    wind_speed            (time, y, x) float32 dask.array<chunksize=(1, 516, 327), meta=np.ndarray>

In [ ]:
portugal = geopandas.read_file('/content/drive/MyDrive/data/tq087tk3612.shp', crs="epsg:4326")

In [ ]:
portugal

,id_0,iso,name_engli,name_iso,name_fao,name_local,name_obsol,name_varia,name_nonla,name_frenc,name_spani,name_russi,name_arabi,name_chine,waspartof,contains,sovereign,iso2,www,fips,ison,validfr,validto,pop2000,sqkm,popsqkm,unregion1,unregion2,developing,cis,transition,oecd,wbregion,wbincome,wbdebt,wbother,ceeac,cemac,ceplg,comesa,eac,ecowas,igad,ioc,mru,sacu,uemoa,uma,palop,parta,cacm,eurasec,agadir,saarc,asean,nafta,gcc,csn,caricom,eu,can,acp,landlocked,aosis,sids,islands,ldc,geometry
0,182.0,PRT,Portugal,PORTUGAL,Portugal,Portugal,NaN,Azores|Madeira,NaN,Portugal,Portugal,??????????,????????,???,NaN,NaN,Portugal,PT,NaN,PO,620.0,Unknown,Present,10015505.0,92098.273,108.748022,Southern Europe,Europe,2.0,0.0,0.0,1.0,NaN,High income: OECD,Debt not classified,EMU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"MULTIPOLYGON (((-16.05835 30.03020, -16.05837 ..."


In [ ]:
ds_portugal.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
ds_portugal.rio.write_crs("epsg:4326", inplace=True)

clipped = ds_portugal.rio.clip(portugal.geometry.apply(mapping), portugal.crs, drop=True, from_disk =True)

In [ ]:
clipped

<xarray.Dataset>
Dimensions:               (x: 327, y: 515, time: 6026)
Coordinates:
  * x                     (x) float64 -9.491 -9.481 -9.471 ... -6.218 -6.208
  * y                     (y) float64 42.14 42.13 42.12 ... 36.99 36.98 36.97
    spatial_ref           int64 0
  * time                  (time) datetime64[ns] 2006-04-01 ... 2022-09-29
Data variables: (12/29)
    aspect                (y, x) float32 dask.array<chunksize=(515, 327), meta=np.ndarray>
    burned_areas          (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    curvature             (y, x) float32 dask.array<chunksize=(515, 327), meta=np.ndarray>
    d2m                   (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    dem                   (y, x) float32 dask.array<chunksize=(515, 327), meta=np.ndarray>
    ignition_points       (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    ...                    ...
    sp                    (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    ssrd                  (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    t2m                   (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    tp                    (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    wind_direction        (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>
    wind_speed            (time, y, x) float32 dask.array<chunksize=(1, 515, 327), meta=np.ndarray>

In [ ]:
# Dataset of areas where wildfires have happened.
# This csv file is located at https://github.com/Aser-Abdelfatah/Portugal-Wildfire-Research/tree/main/Portugal_Dataset_Extraction/data
years = pd.read_csv('/content/drive/MyDrive/data/true.csv', index_col=None)
years = years.drop(columns=[ 'Unnamed: 0'])
years

,x,y,time,aspect,burned_areas,curvature,d2m,dem,ignition_points,lai,lc_agriculture,lc_forest,lc_grassland,lc_settlement,lc_shrubland,lc_sparse_vegetation,lc_water_bodies,lc_wetland,lst_day,lst_night,ndvi,population,rh,roads_distance,slope,smi,sp,ssrd,t2m,tp,wind_direction,wind_speed,year,month,day,day_of_the_year,cos_day_of_the_year,sin_day_of_the_year,x_int,y_int,repetitions
0,-9.480614,38.757993,2018-10-05,218.323290,1.0,-13973.8610,NaN,22.818422,0.0,NaN,0.061867,0.358775,0.000000,0.000000,0.034659,0.000000,0.441838,0.102861,NaN,NaN,NaN,1.502658,NaN,0.000,1.558401,0.13765,NaN,NaN,NaN,NaN,NaN,NaN,2018,10,5,278,0.073095,-0.997325,-948,3875,1
1,-9.480614,38.768062,2018-10-05,238.144700,1.0,4664.3843,NaN,212.681440,0.0,3.7,0.222479,0.501165,0.000000,0.037123,0.201721,0.000000,0.000000,0.037511,301.22000,291.50,0.5433,54.910294,NaN,0.172,1.557888,0.13765,NaN,NaN,NaN,NaN,NaN,NaN,2018,10,5,278,0.073095,-0.997325,-948,3876,1
2,-9.470545,38.727786,2018-10-05,285.109130,1.0,-2931.3470,NaN,22.934958,0.0,NaN,0.094327,0.289674,0.000000,0.299544,0.000000,0.150371,0.166084,0.000000,302.18000,289.72,0.3559,20.905184,NaN,0.000,1.537272,0.15544,NaN,NaN,NaN,NaN,NaN,NaN,2018,10,5,278,0.073095,-0.997325,-947,3872,1
3,-9.470545,38.737855,2018-10-05,258.798920,1.0,-5164.5044,NaN,18.108190,0.0,NaN,0.055267,0.467525,0.110535,0.000000,0.000000,0.087259,0.279414,0.000000,NaN,289.94,NaN,4.693551,NaN,0.000,1.542442,0.15544,NaN,NaN,NaN,NaN,NaN,NaN,2018,10,5,278,0.073095,-0.997325,-947,3873,1
4,-9.470545,38.747924,2018-10-05,234.107250,1.0,-6697.9670,NaN,34.618267,0.0,NaN,0.077921,0.610518,0.012849,0.000000,0.000000,0.000000,0.298713,0.000000,NaN,NaN,NaN,1.748590,NaN,0.000,1.553220,0.13765,NaN,NaN,NaN,NaN,NaN,NaN,2018,10,5,278,0.073095,-0.997325,-947,3874,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34564,-6.218267,41.547098,2013-08-21,106.000244,1.0,-7686.9326,283.89166,656.406300,0.0,0.9,0.221780,0.177264,0.000000,0.000000,0.600956,0.000000,0.000000,0.000000,314.25998,294.18,0.3945,0.017831,0.151507,4.148,1.541004,0.20800,93184.266,13137732.0,304.14734,7.450581e-09,267.504880,4.061390,2013,8,21,233,-0.645348,-0.763889,-621,4154,2
34565,-6.218267,41.547098,2017-06-19,106.000244,1.0,-7686.9326,287.92040,656.406300,0.0,1.0,0.221780,0.177264,0.000000,0.000000,0.600956,0.000000,0.000000,0.000000,NaN,NaN,0.4800,0.017911,0.222242,4.148,1.541004,0.32760,93423.055,14176531.0,306.38330,1.367955e-03,356.901200,3.382208,2017,6,19,170,-0.976938,0.213521,-621,4154,2
34566,-6.218267,41.557167,2013-08-21,105.252884,1.0,-2599.2126,283.87665,674.455300,0.0,1.3,0.560966,0.105398,0.000000,0.000000,0.333636,0.000000,0.000000,0.000000,313.75998,293.74,0.4620,0.000000,0.153281,3.108,1.551348,0.20800,93163.390,13138056.0,304.10320,7.450581e-09,264.396420,4.077430,2013,8,21,233,-0.645348,-0.763889,-621,4155,2
34567,-6.218267,41.557167,2017-06-19,105.252884,1.0,-2599.2126,287.89407,674.455300,0.0,1.5,0.560966,0.105398,0.000000,0.000000,0.333636,0.000000,0.000000,0.000000,NaN,NaN,0.5225,0.000000,0.223632,3.108,1.551348,0.32760,93403.210,14168242.0,306.36700,1.401108e-03,9.063126,3.422222,2017,6,19,170,-0.976938,0.213521,-621,4155,2


# Extraction of Climate Features


In [ ]:
# finds the index coressponding to x an y cooridnate
x_arr = np.asarray(clipped['x'])
y_arr = np.asarray(clipped['y'])

def findy(y):
  return np.where(np.isclose(y_arr, y, 0.0001))[0][0]

def findx(x):
  return np.where(np.isclose(x_arr, x, 0.0001))[0][0]

In [ ]:
# calculates mean over time
clipped_mean = clipped.mean(dim='time', skipna=True)
clipped_mean

In [ ]:
d2m_mean = np.asarray(clipped_mean['d2m'])
wind_speed_mean = np.asarray(clipped_mean['wind_speed'])
wind_direction_mean = np.asarray(clipped_mean['wind_direction'])
lst_night_mean = np.asarray(clipped_mean['lst_night'])
t2m_mean = np.asarray(clipped_mean['t2m'])
tp_mean = np.asarray(clipped_mean['tp'])
sp_mean = np.asarray(clipped_mean['sp'])
ssrd_mean = np.asarray(clipped_mean['ssrd'])
rh_mean = np.asarray(clipped_mean['rh'])
lst_day_mean = np.asarray(clipped_mean['lst_day'])

In [ ]:
#placeholder
years['d2m_mean'] = 0
years['wind_speed_mean'] = 0
years['wind_direction_mean'] = 0
years['lst_night_mean'] = 0
years['t2m_mean'] = 0
years['tp_mean'] = 0
years['sp_mean'] = 0
years['ssrd_meean'] = 0
years['rh_mean'] = 0
years['lst_day_mean'] = 0

In [ ]:
for index1, row1 in years.iterrows():
  years.at[index1, 'd2m_mean'] = d2m_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'wind_speed_mean'] = wind_speed_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'wind_direction_mean'] = wind_direction_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'lst_night_mean'] = lst_night_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 't2m_mean'] = t2m_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'tp_mean'] = tp_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'sp_mean'] = sp_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'ssrd_mean'] = ssrd_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'rh_mean'] = rh_mean[findy(row1['y'])][findx(row1['x'])]
  years.at[index1, 'lst_day_mean'] = lst_day_mean[findy(row1['y'])][findx(row1['x'])]

In [ ]:
years.to_csv('/content/drive/MyDrive/data/climate_data.csv')